In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

from IPython.display import clear_output
from datetime import datetime

In [2]:
import pandas as pd

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./1.category.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['1st Category No'] = input_['1st Category No'].astype(int)
input_['2nd Category No'] = input_['2nd Category No'].astype(int)
input_['3rd Category No'] = input_['3rd Category No'].astype(int)

print(f'总数量：{len(input_)}')
print()

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)

# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error

    while not work.empty():
        a = work.get_nowait()

        # = = = = = = = = = = = = = = =

        crawler_status = 'error'

        # = = = = = = = = = = = = = = =

        import requests

        import sys
        sys.path.append('../00.Tools')
        from crawler_configuration import get_header, get_proxy

        from bs4 import BeautifulSoup
        from lxml import etree

        try:
            df_temp, page = pd.DataFrame(), 1
            while True:
                page_url, b = f'''{input_.loc[a, '3rd Category Url']}?sort=alphaasc&limit=96&page={page}''', 0

                while True:
                    try:

                        while True:
                            b += 1
                            try:
                                resp = requests.get(page_url,
                                                    headers=get_header(),
                                                    proxies=get_proxy(),
                                                    timeout=(10, 10))
                                break
                            except:
                                continue

                        if resp.status_code == 200:
                            soup = BeautifulSoup(resp.text, 'lxml')
                            html = etree.HTML(str(soup))

                            # = = = = = = = = = = = = = = =

                            if input_.loc[a, '3rd Category'] in html.xpath('//title/text()')[0]:
                                list_url = html.xpath('//h4[@class="card-title NoMargin"]/a/@href')

                                df_temp_temp = pd.DataFrame({'1st Category No': input_.loc[a, '1st Category No'],
                                                             '1st Category': input_.loc[a, '1st Category'],
                                                             '1st Category Url': input_.loc[a, '1st Category Url'],
                                                             '2nd Category No': input_.loc[a, '2nd Category No'],
                                                             '2nd Category': input_.loc[a, '2nd Category'],
                                                             '2nd Category Url': input_.loc[a, '2nd Category Url'],
                                                             '3rd Category No': input_.loc[a, '3rd Category No'],
                                                             '3rd Category': input_.loc[a, '3rd Category'],
                                                             '3rd Category Url': input_.loc[a, '3rd Category Url'],
                                                             'Page': page,
                                                             'Page Url': page_url,
                                                             'Part No': [page*96-95+i for i in range(len(list_url))],
                                                             'Url': list_url})

                                df_temp = pd.concat([df_temp, df_temp_temp], ignore_index=True).fillna('')

                                crawler_status = 'ok'

                                # = = = = = = = = = = = = = = =

                                break
                    except:
                        continue

                print(f'''[尝试次数：{b}] - {input_.loc[a, '1st Category No']}.{input_.loc[a, '1st Category']} > {input_.loc[a, '2nd Category No']}.{input_.loc[a, '2nd Category']} > {input_.loc[a, '3rd Category No']}.{input_.loc[a, '3rd Category']} > {page}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

                if not html.xpath('//li[@class="pagination-item pagination-item--next"]'):
                    break
                else:
                    page += 1

            # = = = = = = = = = = = = = = =

            if not df_temp.empty:
                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')
                crawler_status = 'ok'
            else:
                raise
        except:
            df_temp = pd.DataFrame([{'1st Category No': input_.loc[a, '1st Category No'],
                                     '1st Category': input_.loc[a, '1st Category'],
                                     '1st Category Url': input_.loc[a, '1st Category Url'],
                                     '2nd Category No': input_.loc[a, '2nd Category No'],
                                     '2nd Category': input_.loc[a, '2nd Category'],
                                     '2nd Category Url': input_.loc[a, '2nd Category Url'],
                                     '3rd Category No': input_.loc[a, '3rd Category No'],
                                     '3rd Category': input_.loc[a, '3rd Category'],
                                     '3rd Category Url': input_.loc[a, '3rd Category Url']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(12):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['1st Category No', '2nd Category No', '3rd Category No', 'Page', 'Part No'],
                                                ascending=[True, True, True, True, True],
                                                ignore_index=True)
    output_correct.loc[0, 'Part No'] = 1
    for i in range(1, len(output_correct)):
        if output_correct.loc[i, '1st Category No'] == output_correct.loc[i-1, '1st Category No'] and output_correct.loc[i, '2nd Category No'] == output_correct.loc[i-1, '2nd Category No'] and output_correct.loc[i, '3rd Category No'] == output_correct.loc[i-1, '3rd Category No']:
            output_correct.loc[i, 'Part No'] = output_correct.loc[i-1, 'Part No'] + 1
        else:
            output_correct.loc[i, 'Part No'] = 1
    output_correct.to_excel(f'''./2.url-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
if not output_error.empty:
    output_error = output_error.sort_values(by=['1st Category No', '2nd Category No', '3rd Category No'],
                                            ascending=[True, True, True],
                                            ignore_index=True)
    output_error.to_excel('./url_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：112

[尝试次数：1] - 1.Brakes > 2.Brake Boosters & Pumps > 1.Electronic Brake Assembly > 1
[剩余数量：100] - [当前时间：19:57:00]

[尝试次数：1] - 1.Brakes > 2.Brake Boosters & Pumps > 5.Power Brake Booster > 1
[剩余数量：99] - [当前时间：19:57:01]

[尝试次数：1] - 1.Brakes > 2.Brake Boosters & Pumps > 4.Electronic Brake Pump & Motor > 1
[剩余数量：99] - [当前时间：19:57:01]

[尝试次数：1] - 1.Brakes > 2.Brake Boosters & Pumps > 6.Power Brake Booster Hydraulic Motor Pump > 1
[剩余数量：98] - [当前时间：19:57:02]

[尝试次数：1] - 1.Brakes > 1.ABS Components > 3.ABS Pump & Motor Assembly > 1
[剩余数量：97] - [当前时间：19:57:02]

[尝试次数：1] - 1.Brakes > 2.Brake Boosters & Pumps > 2.Electronic Brake Booster Vacuum Pump > 1
[剩余数量：96] - [当前时间：19:57:02]

[尝试次数：1] - 1.Brakes > 2.Brake Boosters & Pumps > 7.Vacuum Pump > 1
[剩余数量：95] - [当前时间：19:57:02]

[尝试次数：1] - 1.Brakes > 3.Disc Brakes > 1.Air Disc Brake Caliper > 1
[剩余数量：94] - [当前时间：19:57:03]

[尝试次数：1] - 1.Brakes > 1.ABS Components > 2.ABS Hydraulic Assembly > 1
[剩余数量：93] - [当前时间：19:57:03]

[尝试次数：1] - 1.Brakes > 2